# PLEASE USE `Cell => Run All` ON THIS NOTEBOOK (MENU OPTION ABOVE).

# THIS NOTEBOOK WILL TAKE 20-30 MINUTES TO COMPLETE SUCCESSFULLY.

# PLEASE BE PATIENT

In [1]:
import sys
print('Python Version %s' % sys.version)

Python Version 3.6.10 |Anaconda, Inc.| (default, Jan  7 2020, 21:14:29) 
[GCC 7.3.0]


In [2]:
!pip list

Package                            Version            
---------------------------------- -------------------
alabaster                          0.7.12             
anaconda-client                    1.7.2              
anaconda-project                   0.8.3              
argh                               0.26.2             
asn1crypto                         1.3.0              
astroid                            2.4.2              
astropy                            4.0                
atomicwrites                       1.3.0              
attrs                              19.3.0             
Automat                            20.2.0             
autopep8                           1.4.4              
autovizwidget                      0.16.0             
awscli                             1.18.169           
Babel                              2.8.0              
backcall                           0.1.0              
backports.shutil-get-terminal-size 1.0.0              
bcrypt    

In [3]:
%%bash
conda list

# packages in environment at /home/ec2-user/anaconda3/envs/python3:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
alabaster                 0.7.12                   py36_0  
anaconda-client           1.7.2                    py36_0  
anaconda-project          0.8.4                      py_0  
argh                      0.26.2                   py36_0  
argparse                  1.4.0                    pypi_0    pypi
asn1crypto                1.3.0                    py36_0  
astroid                   2.4.2                    pypi_0    pypi
astropy                   4.0              py36h7b6447c_0  
atomicwrites              1.3.0                    py36_1  
attrs                     19.3.0                     py_0  
automat                   20.2.0                   pypi_0    pypi
autopep8                  1.4.4                      py_0  
autovizwidget             0.16.0                   pypi_0    pypi

# 1. Install the CLI tools

_Note:  Make sure the SageMaker role has access to create EKS Cluster._

In [4]:
%%bash

source ~/.bash_profile

pip install awscli --upgrade --user

curl --silent --location "https://github.com/weaveworks/eksctl/releases/latest/download/eksctl_$(uname -s)_amd64.tar.gz" | tar xz -C /tmp

sudo mv /tmp/eksctl /usr/local/bin

eksctl version

curl --location -o ./kubectl https://amazon-eks.s3.us-west-2.amazonaws.com/1.18.8/2020-09-18/bin/linux/amd64/kubectl
#curl --location -o ./kubectl https://amazon-eks.s3.us-west-2.amazonaws.com/1.15.10/2020-02-22/bin/linux/amd64/kubectl

chmod +x ./kubectl

sudo mv ./kubectl /usr/local/bin

kubectl version --short --client

curl -o aws-iam-authenticator https://amazon-eks.s3.us-west-2.amazonaws.com/1.18.8/2020-09-18/bin/linux/amd64/aws-iam-authenticator
#curl -o aws-iam-authenticator https://amazon-eks.s3.us-west-2.amazonaws.com/1.15.10/2020-02-22/bin/linux/amd64/aws-iam-authenticator

chmod +x ./aws-iam-authenticator

sudo mv aws-iam-authenticator /usr/local/bin

aws-iam-authenticator version

sudo yum -y install jq gettext

for command in kubectl jq envsubst
  do
    which $command &>/dev/null && echo "$command in path" || echo "$command NOT FOUND"
  done

echo "Completed"

0.32.0
Client Version: v1.18.8-eks-7c9bda
{"Version":"v0.5.0","Commit":"1cfe2a90f68381eacd7b6dcfa2bf689e76eb8b4b"}
Loaded plugins: dkms-build-requires, priorities, update-motd, upgrade-helper,
              : versionlock
Package jq-1.5-1.2.amzn1.x86_64 already installed and latest version
Package gettext-0.18.1.1-9.1.11.amzn1.x86_64 already installed and latest version
Nothing to do
kubectl in path
jq in path
envsubst in path
Completed


You should consider upgrading via the '/home/ec2-user/anaconda3/bin/python -m pip install --upgrade pip' command.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57.2M  100 57.2M    0     0  45.4M      0  0:00:01  0:00:01 --:--:-- 45.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33.6M  100 33.6M    0     0  50.0M      0 --:--:-- --:--:-- --:--:-- 50.0M


# 2. Configure the Environment Variables

In [5]:
%%bash

source ~/.bash_profile

export AWS_REGION=$(aws configure get region)
echo "export AWS_REGION=${AWS_REGION}" | tee -a ~/.bash_profile

export AWS_CLUSTER_NAME=workshop
echo "export AWS_CLUSTER_NAME=${AWS_CLUSTER_NAME}" | tee -a ~/.bash_profile

echo "Completed"

export AWS_REGION=us-west-2
export AWS_CLUSTER_NAME=workshop
Completed


# 3. Create the EKS Cluster

# THIS WILL TAKE 20-30 MINUTES.  PLEASE BE PATIENT!

# _If you see `retryable error`s below, this is OK!_

In [6]:
%%bash

###############################
#
# ONLY RUN THIS CELL ONE TIME!!
#
###############################

source ~/.bash_profile

export AWS_CLUSTER_STATUS=$(aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.status" --output text)

if [ -n "$AWS_CLUSTER_STATUS" ]; then
    if [ $AWS_CLUSTER_STATUS == "ACTIVE" ]; then
        echo "Cluster status:  $AWS_CLUSTER_STATUS."
    else
        echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please wait for status:  ACTIVE"
        exit
    fi
else
    # THIS WILL TAKE 20-30 MINUTES.  PLEASE BE PATIENT. 
    eksctl create cluster \
      --name ${AWS_CLUSTER_NAME} \
      --version 1.18 \
      --nodes 4 \
      --node-type c5.xlarge \
      --node-volume-size 100 \
      --region=${AWS_REGION} \
      --zones=${AWS_REGION}a,${AWS_REGION}b \
      --alb-ingress-access
      
    export AWS_CLUSTER_STATUS=$(aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.status" --output text)
    echo "export AWS_CLUSTER_STATUS=${AWS_CLUSTER_STATUS}" | tee -a ~/.bash_profile
    echo "Cluster status:  $AWS_CLUSTER_STATUS"
fi

[ℹ]  eksctl version 0.32.0
[ℹ]  using region us-west-2
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while awaiting headers)) from ec2metadata/GetToken - will retry after delay of 40.990565ms
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while awaiting headers)) from ec2metadata/GetToken - will retry after delay of 103.943946ms
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while awaiting headers)) from ec2metadata/GetToken - will retry after delay of 176.797324ms
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while awai


An error occurred (ResourceNotFoundException) when calling the DescribeCluster operation: No cluster found for name: workshop.


# _Your output will look like this..._

![](../../../img/verify_eks.png)


# _If you see `retryable error`s  ^^ above ^^, this is OK!_

# AFTER 20-30 MINUTES, YOUR CLUSTER WILL BE CREATED.

In [7]:
%%bash

source ~/.bash_profile

export AWS_CLUSTER_STATUS=$(aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.status" --output text)

if [ -n "$AWS_CLUSTER_STATUS" ]; then
    if [ $AWS_CLUSTER_STATUS == "ACTIVE" ]; then
        echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please continue."
    else
        echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please wait for status:  ACTIVE"
        exit
    fi
fi

Cluster status:  ACTIVE.  Please continue.
